[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/redcican/Coursera_Capstone/blob/master/First_Assignment_Neighborhood_Segmentation_and_Clustering.ipynb)

# IBM Coursera Final Project 1: Segmenting and Clustering Neighborhoods in Toronto - First Assignment

## 1. Import the Libraries

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text

Use Beautifulsoup to extract the table content from URL page.

In [0]:
soup = BeautifulSoup(html, 'html.parser')

Find the table, and check the content.

In [0]:
my_table = soup.find('table',{'class':'wikitable sortable'})
my_table

Only find all cells.

In [0]:
cells = my_table.find_all('td')
cells

Define 3 lists for 3 columns.

In [0]:
Postcode = [];
Borough = [];
Neighbourhood = [];

A for loop to iterate all data.

In [0]:
for row in my_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))

Convert the lists to a Pandas Dataframe, and check the first 10 rows.

In [25]:
df = pd.DataFrame({"Postcode":Postcode,
                  "Borough":Borough,
                  "Neighbourhood":Neighbourhood})
df = df[['Postcode','Borough','Neighbourhood']]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Ignore cells with a borough that is **Not assigned**.

In [26]:
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Using Pandas `groupby`function to combine two rows into one row with the neighborhoods separated with a comma.

In [29]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df = df.replace(r'\n','', regex=True) 
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.

In [0]:
for i in range(len(df['Neighbourhood'])):
    if df['Neighbourhood'].iloc[i] == 'Not assigned':
        df['Neighbourhood'].iloc[i] = df['Borough'].iloc[i]

In [41]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


Use the **.shape** method to print the number of rows.

In [42]:
df.shape

(103, 3)